In [46]:
import os

In [47]:
import asyncio
from typing import Optional
from dotenv import load_dotenv

In [48]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

In [49]:
load_dotenv()

True

In [50]:
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    print("LLM initialized Successfully")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None
    

LLM initialized Successfully


In [51]:
# Build 3 Chains

summarize_chain: Runnable = ChatPromptTemplate.from_messages(
                                                              [
                                                                  ("system", "Summarize the following topic concisely"),
                                                                  ("user", "{topic}")
                                                              ]
    
                                                            ) | llm  | StrOutputParser()        





In [52]:
questions_chain: Runnable = ChatPromptTemplate.from_messages(
                                                              [
                                                                  ("system", "Generate 3 interesting questions about the following topic"),
                                                                  ("user", "{topic}")
                                                              ]
    
                                                            ) | llm  | StrOutputParser()        



    

In [53]:
terms_chain: Runnable = ChatPromptTemplate.from_messages(
                                                              [
                                                                  ("system", "Identify 5-10 key terms from the following topic, separated by commas"),
                                                                  ("user", "{topic}")
                                                              ]
    
                                                            ) | llm  | StrOutputParser()        



    

In [54]:
# Build The parallel + SYnthesis Chain

In [55]:
map_chain = RunnableParallel(
                                {
                                    "summary": summarize_chain,
                                    "questions": questions_chain,
                                    "terms": terms_chain,
                                    "topic": RunnablePassthrough()
                                }
        
                             ) 

In [56]:
synthesis_prompt = ChatPromptTemplate.from_messages([
                                                    ("system", 
                                                                 """
                                                                 Based on the following information:
                                                                 Summary: {summary},
                                                                 Questions: {questions},
                                                                 Terms: {terms}
                                                                 generate a comprehensive answer
                                                                 """),
                                                    ("user", "Original topic: {topic}")
    
                                                    ])

In [57]:
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

In [58]:
async def run_parallel_example(topic: str) -> None :
    if not llm:
        print("LLM not initialized")
        return
    try:
        response = await full_parallel_chain.ainvoke(topic)
        print("\n -- FInal response")
        print(response)
    except Exception as e:
        print(f"\n An error occurred during chain execution: {e}")

In [61]:
async def main():
    topic = "The history of space exploration"
    print("Starting Parallel Chain")
    await run_parallel_example(topic)
    print("Done with Parallel Chain")
    

In [62]:
# Cell 10: Jupyter compatibility
import nest_asyncio

# Cell 11: Apply patch
nest_asyncio.apply()

# Cell 12: Run
await main()

Starting Parallel Chain

 -- FInal response
### The History of Space Exploration

#### 1. Cold War Rivalry and Early Milestones

The Cold War rivalry between the United States and the Soviet Union was a catalyst for the rapid advancements in space exploration during the mid-20th century. The launch of Sputnik 1 on October 4, 1957, by the Soviet Union marked the beginning of the Space Race, a competition that fueled significant technological innovations and national pride. This event not only demonstrated Soviet technological prowess but also instigated fears in the U.S. about potential military applications of space technology.

Key milestones during this era included:

- **Yuri Gagarin's Flight (1961)**: The USSR achieved a significant victory with Yuri Gagarin becoming the first human to orbit the Earth, which showcased the capabilities of Soviet space technology and intensified U.S. efforts to catch up.
- **Apollo Program (1960s)**: In response, NASA launched the Apollo program, cul

The provided Python code implements a LangChain application designed for processing a given topic efficiently by leveraging parallel execution. Note that asyncio provides concurrency, not parallelism. It achieves this on a single thread by using an event loop that intelligently switches between tasks when one is idle (e.g., waiting for a network request). This creates the effect of multiple tasks progressing at once, but the code itself is still being executed by only one thread, constrained by Python’s Global Interpreter Lock (GIL).